In [1]:
import cx_Oracle
import pandas as pd
from datetime import timedelta
from datetime import datetime   
import numpy as np
import os
import re

In [2]:
# Initializing database connection
con = cx_Oracle.connect('STC_SETRAG/art@localhost/orcl.eng.engesis.com')
cur = con.cursor()

    # First Query
query=cur.execute('select * from historico_mensagens')
df = pd.DataFrame(query)
    # Second Query

cur.close()
con.close()

In [3]:
df.columns=['HM_ID_HM','HM_LOCO_MSG','HM_PRF_TRM','HM_OBC_MSG','HM_NUM_MSG','HM_TP_MSG','HM_DAT_OBC','HM_DAT_STC', 'HM_TXT_MSG','HM_TXT_TW','HM_LAT_HM','HM_LON_HM','HM_ENV_RCB','HM_MSG_VOZ','HM_NUM_FUN','HM_ID_AUX','HM_SIT_ENV','MT_ID_MT']
df = df.drop(['HM_TP_MSG','HM_TXT_TW','HM_ID_AUX','HM_OBC_MSG','HM_LAT_HM','HM_LON_HM'], axis=1)

df_sent = df.loc[df['HM_ENV_RCB']=='E']
df_received = df.loc[df['HM_ENV_RCB']=='R']

    # Removing the original dataframe
df = pd.DataFrame(None)

In [4]:
month = '2'
year = '2021'
a = timedelta(seconds=300)

In [5]:
df_sent = df_sent.loc[df_sent['HM_DAT_STC']>= year + '-' + month +'-01']
if(month == '12'):     # December
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< year + '-' + str(int(month)+1) +'-01']
    # 2) Filter all messages that contain OBC time
df_sent = df_sent.loc[df_sent['HM_DAT_OBC'].notnull()]
    # 3) Filter messages to ensure that the STC time is smaller (or igual) to the OBC time (avoid negative values)
df_sent = df_sent.loc[df_sent['HM_DAT_STC']<=df_sent['HM_DAT_OBC']]
    # 4) Filter messages that contain locomotive ID
df_sent = df_sent[df_sent['HM_LOCO_MSG'].notnull()]

    # Received DataFrame
    # 1) Filter all messages that did not expired
df_received = df_received.loc[df_received['HM_SIT_ENV']=='E']
    # 2) Filter messages from within the date set by user
df_received = df_received.loc[df_received['HM_DAT_STC']>= year + '-' + month +'-01']
if(month == '12'):     # December
    df_received = df_received.loc[df_received['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_received = df_received.loc[df_received['HM_DAT_STC']< year + '-' + str(int(month)+1) +'-01']
    # 3) Filter all messages that contain OBC time
df_received = df_received.loc[df_received['HM_DAT_OBC'].notnull()]
    # 4) Filter all messages that have a date later than needed (avoiding future date messages)
if(month == '12'):
    df_received = df_received.loc[df_received['HM_DAT_OBC']< str(int(year)+1) +'-1-02']
else:
    df_received = df_received.loc[df_received['HM_DAT_OBC']< year + '-' + str(int(month)+1) +'-02']
    # 5) Filter messages to ensure that the OBC time is smaller (or igual) to the STC time (avoid negative values)
df_received = df_received.loc[df_received['HM_DAT_OBC']<=df_received['HM_DAT_STC']]
    # 6) Filter messages that contain locomotive ID
df_received = df_received[df_received['HM_LOCO_MSG'].notnull()]

In [6]:
totalSentMessages = len(df_sent['HM_ID_HM'])
    # Received
totalReceivedMessages = len(df_received['HM_ID_HM'])
    # Total
totalMessages = totalSentMessages + totalReceivedMessages

In [7]:
    # Sent
df_sent['DELIVERY_TIME'] = df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']
    # Converting to string in order to execute a mathematic operation
df_sent['DELIVERY_TIME'] = df_sent['DELIVERY_TIME']/np.timedelta64(1, 's')
totalSent = sum(df_sent.iloc[:,11])
averageSentDelivery = round(totalSent/totalSentMessages,0)
    # Converting from float to mm:ss
averageSentDelivery = timedelta(seconds=averageSentDelivery)
averageSentDelivery=str(averageSentDelivery)

    # Received
df_received['DELIVERY_TIME'] = df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']
    # Converting to string in order to execute a mathematic operation
df_received['DELIVERY_TIME'] = df_received['DELIVERY_TIME']/np.timedelta64(1, 's')
totalReceived = sum(df_received.iloc[:,11])
averageReceivedDelivery = round(totalReceived/totalReceivedMessages,0)


In [8]:
    # Sent
    # 3) Filter messages that took longer than 5 minutes
df_sent = df_sent.loc[(df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']) >= a]
    # 4) Order the dataframe by HM_ID_HM
df_sent = df_sent.sort_values(by=['HM_ID_HM'])

    #Received
    # 5) Filter messages that took longer than 5 minutes
df_received = df_received.loc[(df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']) >= a]
    # 6) Order the dataframe by HM_ID_HM
df_received = df_received.sort_values(by=['HM_ID_HM'])

In [9]:
#converting it from float to INT
df_sent['MT_ID_MT']=df_sent['MT_ID_MT'].astype('int')

In [10]:
myfile = open("D:\\kenneth\\Documents\\Clients\\SETRAG\\Tickets\\SDSTG_561_Server\\SDSTG_561\\app_STCROUTER.log")
log = myfile.read().splitlines()
myfile.close

<function TextIOWrapper.close()>

In [11]:
log

12 00:15:20,201 [8] INFO  - Receive data[Data: 10-00-34-3F-44-00-02-00-00-00-5D-12-03-20-00-2D-14-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-01-01-BD-75]',
 '[6880] 2021-02-12 00:15:25,136 [6] INFO  - Looking for updates...',
 '[6880] 2021-02-12 00:15:25,138 [6] DEBUG - Nothing new.',
 '[6880] 2021-02-12 00:15:28,707 [11] DEBUG - KEEP-ALIVE - Sending keep-alive message.',
 '[6880] 2021-02-12 00:15:30,139 [6] INFO  - Looking for updates...',
 '[6880] 2021-02-12 00:15:30,141 [6] DEBUG - Nothing new.',
 '[6880] 2021-02-12 00:15:30,203 [8] INFO  - Receive data[Data: 10-00-34-3F-44-00-02-00-00-00-5D-12-03-20-00-2D-14-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-01-01-BD-75]',
 '[6880] 2021-02-12 00:15:35,143 [6] INFO  - Looking for updates...',
 '[6880] 2021-02-12 00:15:35,145 [6] DEBUG - Nothing new.',
 '[6880] 2021-02-12 00:15:

In [72]:
# Regex used to match relevant loglines 
line_regex = re.compile("Updating STC message ID: 1858487 to 'E'*")

# Output file, where the matched loglines will be copied to
output_filename = os.path.normpath("parsed_lines.log")
# Overwrites the file, ensure we're starting out with a blank file
with open(output_filename, "w") as out_file:
    out_file.write("")

# Open output file in 'append' mode
with open(output_filename, "a") as out_file:
    # Open input file in 'read' mode
    with open("app_STCROUTER.log", "r") as in_file:
        # Loop over each log line
        for line in in_file:
            # If log line matches our regex, print to console, and output file
            if (line_regex.search(line)):
                print (line)
                file_line = line
                out_file.write(line)

[6880] 2021-02-12 09:27:49,914 [144] INFO  - Locomotive: 4314 -> Updating STC message ID: 1858487 to 'E'



In [80]:
file_line = file_line.rstrip()
line_number = log.index(file_line)
line_number

27943

In [81]:
log[line_number-2]

'[6880] 2021-02-12 09:27:49,853 [8] INFO  - Receive data[Data: 10-00-0C-02-24-10-DA-44-00-02-B6-11-20-20-40-00-0F-00-00-4E-8D-CD-02]'